In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/home/sunho/Documents/dev/gorani-reader/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('App Name')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters


In [5]:
import pyspark.sql.functions as F
from pyspark.ml.feature import CountVectorizer, Normalizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix

from gorani.spark import read_data_all, write_data
from gorani.utils import sparse_to_array

SIMILARITY_TYPE = 'cosine'

words = read_data_all(spark, 'words', cache = True)
books = read_data_all(spark, 'books')

cv = CountVectorizer()\
    .setInputCol('content')\
    .setOutputCol('tf')\
    .setVocabSize(words.count())
cv = cv.fit(books)

nor = Normalizer()\
    .setInputCol('tf')\
    .setOutputCol('norm')

# normalized book-wordcount matrix
tf_mat = nor.transform(cv.transform(books))

# convert to SparseMatrix to BlockMatrix
mat = IndexedRowMatrix(
tf_mat.select("id", "norm")\
    .rdd.map(lambda row: IndexedRow(row['id'], row['norm'].toArray())))\
    .toBlockMatrix()

# cosine similarity
sim_mat_df = mat.multiply(mat.transpose())\
    .toIndexedRowMatrix()\
    .rows.toDF()

# change schema
df = sim_mat_df.select(F.col('index').alias('id'), F.posexplode(sparse_to_array('vector')))\
    .select('id', F.col('pos').alias('other_id'), F.col('col').alias('value'))\
    .withColumn('type', F.lit(ComputeCosineSimilarity.SIMILARITY_TYPE))\
    .where('id != 0 AND other_id != 0')

write_data('book_similarity', df)

IllegalArgumentException: 'CountVectorizer_7bfd54a26c2b parameter vocabSize given invalid value 0.'